# Calculo de las metricas de un json

In [1]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Lectura del documento

In [2]:
# Ruta de los archivos con los datos obtenidos por el "Tracking system"
path = '../DataTracker'

In [3]:
def loadJsonData(path):
    jsonData = []
    
    # Obtener lista de archivos en la carpeta
    jsonFiles = os.listdir(path)
    
    # Recorrer y filtrar solo los archivos JSON
    for file in jsonFiles:
        if file.endswith('.json'):
            ruta_archivo = os.path.join(path, file)
            
            # Leer el contenido del archivo JSON
            with open(ruta_archivo, 'r', encoding='utf-8') as f:
                info = json.load(f)
                jsonData.extend(info)
    
    return jsonData

In [4]:
# Leer los archivos JSON y obtener su información
data = loadJsonData(path)

# Se ordena la información del documento en base el timestamp
sorted_data = sorted(data, key=lambda x: x['timestamp'])

### Parseo de Eventos

In [5]:
# Numero total de eventos
num_events = len(sorted_data)
# Indice de eventos
index = 0
sessions = []
sesion = 0
test_list = []
test = 0
pos_list = []

In [6]:
def parseEvent(event):
    global test_list
    global pos_list
    global sesion
    global test
    global sessions
    if event['typeEvent'] == "LoginEvent":
        test = 0
        #print("\nInicio de sesion: ", sesion)
    elif event['typeEvent'] == "LogoutEvent":
        sessions.append(test_list.copy())
        test_list.clear()
        #print("\nFin de sesion: ", sesion)
        sesion += 1
    # Empieza el test
    elif event['typeEvent'] == "StartTestEvent":
       print("\nInicio de test: ", test)
    # Acaba el test
    elif event['typeEvent'] == "EndTestEvent":
        test_list.append(pos_list.copy())
        pos_list.clear()
        #print("\nFin de test: ", test)
        test += 1
    #Evento de movimeinto de un bot
    elif event['typeEvent'] == "BotPositionEvent":
        pos_list.append(event)
    return  True

In [7]:
# Recorro todos los eventos generados
while index < num_events:
    currentEvent = sorted_data[index]
    consumeEvent = parseEvent(currentEvent)
    index += 1


Inicio de test:  0

Inicio de test:  0

Inicio de test:  0


### Mapa de calor con las posiciones

In [8]:
# Guardar la imagen del mapa
img = plt.imread("images/map2.png")

# Dimensiones del mapa
map_size = 200
#Divido el mapa en casillas
cellsize = 50

# Función para calcular el % de mapa recorrido
def calculate_visited_area(positions):
    x_cells = np.linspace(0, map_size, cellsize+1)
    y_cells = np.linspace(0, map_size, cellsize+1)
    visited_cells = set()
    
    for _, row in positions.iterrows():
        x, y = row['posX'], row['posZ']
        x = max(0, min(map_size, x)) # Me aseguro que la coordenada este dentro de los limites del mapa
        y = max(0, min(map_size, y)) # Me aseguro que la coordenada este dentro de los limites del mapa
        #print(f'x: {x}')
        x_bin = np.digitize(x, x_cells) - 1
        #print(f'indice: {x_bin}')
        y_bin = np.digitize(y, y_cells) - 1
        #Me aseguro que este dentro de los limites de la grid
        if 0 <= x_bin < cellsize and 0 <= y_bin < cellsize:
            visited_cells.add((x_bin, y_bin))
    
    n_cells_visited = len(visited_cells) #Numero de casillas visitadas
    gridsize = cellsize * cellsize #Total ed casillas
    percentage_visited = (n_cells_visited / gridsize) * 100
    return percentage_visited

for i, currentSession in enumerate(sessions):
    for j, currentTest in enumerate(currentSession):
        dfTest = pd.DataFrame(currentTest)
        
        # Calcular el porcentaje del área recorrida
        percentage_visited = calculate_visited_area(dfTest)
        
        # Imprimir el porcentaje del área recorrida
        print(f'Session {i}, Test {j}: {percentage_visited:.2f}% del mapa cubierto')
        
        # Definir las dimensiones del mapa
        fig, ax = plt.subplots(figsize=(10, 10))  # Tamaño en pulgadas de la imagen cuando se guarde
        ax.imshow(img)
        
        # Dibujar el mapa de calor
        dfTest.plot.hexbin(fig=fig, ax=ax, x="posX", y="posZ", reduce_C_function=sum, gridsize=(50, 50), extent=[0, map_size-1, 0, map_size-1], alpha=0.5, cmap='Reds')
        
        # Configurar las etiquetas de los ejes
        ax.set_xticks(range(0, map_size+1, 50))
        ax.set_yticks(range(0, map_size+1, 50))
        
        # Guardar el gráfico como imagen
        fig.savefig(f'heatmaps/map1/heatmapSession{i}Test{j}.png')
        plt.close(fig)

Session 0, Test 0: 59.72% del mapa cubierto
Session 1, Test 0: 69.92% del mapa cubierto
Session 2, Test 0: 89.72% del mapa cubierto
